In [8]:
import time
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value
import numpy as np
import pandas as pd

time_start=time.time()

# Load data
data_dir = '../data/raw/'
filename = 'g14.txt2000.txt'
import_dir = data_dir + filename

#nedges = 1500
#fullfile = np.loadtxt(import_dir, usecols=(0, 1), dtype=int)
#data_import = fullfile[:int(nedges)+1,:]
data_import = np.loadtxt(import_dir, usecols=(0, 1), dtype=int)
npoints = data_import[0, 0] + 1
nedges = data_import[0, 1]
data = np.delete(data_import, 0, axis=0)
w = np.loadtxt(import_dir, usecols=(2), skiprows=1, dtype=int)

# Create the problem instance
problem = LpProblem("Binary_Optimization_Problem", LpMaximize)

# Define binary decision variables
x = LpVariable.dicts("x", range(npoints), cat='Binary')
e = LpVariable.dicts("e", range(nedges), cat='Binary')

# Objective function: maximize the sum of weights of selected edges
problem += lpSum(e[i] * w[i] for i in range(nedges))

# Constraints
for i, (col1, col2) in enumerate(data):
    # Adjust for 1-based indexing if necessary
    if col1 < 0 or col1 >= npoints or col2 < 0 or col2 >= npoints:
        print(f"Invalid indices: col1={col1}, col2={col2} for npoints={npoints}")
        continue  # Skip this iteration if indices are invalid
    problem += x[col1] + x[col2] >= e[i]  # xi + xj >= eij
    problem += x[col1] + x[col2] + e[i] <= 2  # 2 - (xi + xj) >= eij

# Solve the problem
problem.solve()

objective_value = ["Results", value(problem.objective)]
print("Objective Function Value:", objective_value)

# Prepare results for output
results = {
    "Variable": [v.name for v in problem.variables()],
    "Value": [v.varValue for v in problem.variables()],
}

# Create a DataFrame
results_df = pd.DataFrame(np.vstack((pd.DataFrame(results), objective_value)))

# Output results to CSV
output_name = f'../data/output/Pulp/output_{filename}'
results_df.to_csv(output_name, index=False)

# End the timer
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - time_start

print(f"Optimization complete. Results saved to: {output_name}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Objective Function Value: ['Results', 1716.0]
Optimization complete. Results saved to: ../data/output/Pulp/output_g14.txt2000.txt
Elapsed time: 118.06 seconds


In [6]:
print(f"Using solver: {problem.solver}")

Using solver: <pulp.apis.coin_api.PULP_CBC_CMD object at 0x0000018D28543BE0>
